# IN3050/IN4050 - Week 2
## Representations

### 1. ![Naming_Question](EA_Terms.png)

Name the terms shown in the picture above.

# Add your solution here
### A: Locus
### B: Allele
### C: Gene
### D: Genotype
### E: Phenotype

### 2. Mention some of the most common representations of genomes.

Genomes are represented in binary, integers, permutation, tree, and real numbers

### 3. Perform a mutation operation on the representations given below.

binary = $[1, 0, 1, 1]$;
integer = $[4, 2, 4, 1]$;
real_valued = $[2.53, 1.42, 3.14, 1.68]$;
permutation = $[3, 4, 1, 2]$

In [20]:
b   = [1, 0, 1, 1]
b_p = [1, 1, 1, 1]

i   = [4, 2, 4, 1]
i_p   = [0, 2, 4, 1]

rv   = [2.53, 1.42, 3.14, 1.68] 
rv_p = [3.14, 1.68, 2.53, 1.42]

### 4. Given the sequences (2,4,7,1,3,6,8,9,5) and (5,9,8,6,2,4,1,3,7). Implement these algorithms to create a new pair of solutions: 

#### a. Partially mapped crossover (PMX)

In [84]:
# Add your solution here
import numpy as np
from typing import Iterable
np.random.seed(42)   

def pmx(p1: Iterable, p2: Iterable):
    def procedure(p1: Iterable, p2: Iterable):
        p1, p2 = np.array(p1), np.array(p2)
        # Step 1
        idx1 = np.random.randint(0, len(p1))
        idx2 = np.random.randint(0, len(p1))
        
        while idx1 == idx2:
            idx2 = np.random.randint(0, len(p1))
        
        if idx1 > idx2:
            idx2, idx1 = idx1, idx2
        
        segment = slice(idx1, idx2+1)
        segment = slice(3,7) # Sanity check (lecture 3, slide 56)
        
        child = np.zeros(len(p1), dtype=int)
        child[segment] = p1[segment]
        
        # Step 2
        for gene1 in p1[segment]:
            gene1_idx = np.where(p1 == gene1)
            gene2 = p2[gene1_idx]
            
            if gene2 in child:
                continue
            
            gene1_in_p2_idx = np.where(p2 == gene1)
            while gene1 in p2[segment]:
                gene1 = p1[gene1_in_p2_idx]
                gene1_in_p2_idx = np.where(p2 == gene1)
                    
            child[gene1_in_p2_idx] = gene2
            
        # Step 3
        for idx, gene in enumerate(child):
            if gene == 0:
                child[idx] = p2[idx]
                
        return child    
    
    child1 = procedure(p1, p2)
    child2 = procedure(p2, p1)
    
    return child1, child2

p1 = (2,4,7,1,3,6,8,9,5)
p2 = (5,9,8,6,2,4,1,3,7)

# Sanity check (lecture 3, slide 56)
p1 = (1,2,3,4,5,6,7,8,9)
p2 = (9,3,7,8,2,6,5,1,4)

child1, child2 = pmx(p1, p2)
print(f'{child1 = }')
print(f'{child2 = }')

child1 = array([9, 3, 2, 4, 5, 6, 7, 1, 8])
child2 = array([1, 7, 3, 8, 2, 6, 5, 4, 9])


#### b. Order crossover

In [82]:
# Add your solution here
import numpy as np
from typing import Iterable
from time import time
np.random.seed(42)   

def order_crossover(p1: Iterable, p2: Iterable, _test: bool = False) -> tuple[np.ndarray, np.ndarray]:
    def procedure(p1: Iterable, p2: Iterable):
        p1, p2 = np.array(p1), np.array(p2)
        n = len(p1)
        # Step 1
        if _test:
            idx1 = 3
            idx2 = 6
        else:
            idx1 = np.random.randint(0, n)
            idx2 = np.random.randint(0, n)
            
            # Make sure idx1 != idx2
            while idx1 == idx2:
                idx2 = np.random.randint(0, n)
            
            # Make sure idx1 alwyas refers to the smaller index
            if idx1 > idx2:
                idx2, idx1 = idx1, idx2
            
        segment = slice(idx1, idx2+1)
        
        child = np.zeros(n, dtype=int)
        child[segment] = p1[segment]
        
        # Step 2
        next_available_index = idx2 + 1
        for i in range(1,n+1):
            gene = p2[(idx2 + i) % n]
            if gene not in child:
                child[next_available_index % n] = gene
                next_available_index += 1
        
        return child 
    
    child1 = procedure(p1, p2)
    child2 = procedure(p2, p1)
    
    return child1, child2

# Sanity check (lecture 3, slide 61)
p1 = (1,2,3,4,5,6,7,8,9)
p2 = (9,3,7,8,2,6,5,1,4)

child1, child2 = order_crossover(p1, p2, _test=True)
print(f'{child1 = }')
print(f'{child2 = }')

child1 = array([3, 8, 2, 4, 5, 6, 7, 1, 9])
child2 = array([3, 4, 7, 8, 2, 6, 5, 9, 1])


#### c. Cycle crossover

In [40]:
import numpy as np
from typing import Iterable


def cycle_crossover(p1: Iterable, p2: Iterable):
    p1, p2 = np.array(p1), np.array(p2)
    
    # Step 1: Find Cycles
    cycles = []
    for i in range(len(p1)):
        allele = p1[i]
        allele_adj = p2[i]
        idx = np.where(p1 == allele_adj)[0][0]
        
        # Check for element already in a cycle
        visited = False
        for cycle in cycles:
            if idx in cycle:
                visited = True
                break
        
        if visited:
            continue
        
        # Find cycle
        ## First element
        cycle = []
        cycle.append(idx)
        
        first = allele 
        allele = p1[idx]
        allele_adj = p2[idx]
        ## The rest of the elements
        while first != allele:
            idx = np.where(p1 == allele_adj)[0][0]
            cycle.append(idx)   
            allele = p1[idx]
            allele_adj = p2[idx]  
        
        cycles.append(cycle)  
    
    
    child1 = np.zeros(len(p1), dtype=int)
    child2 = np.zeros(len(p2), dtype=int)
    
    # Step 2: Alternating parents
    for cycle in cycles:
        for idx in cycle:
            child1[idx] = p1[idx]
            child2[idx] = p2[idx]
            
        p1, p2 = p2, p1
        
    return child1, child2 

p1 = (2,4,7,1,3,6,8,9,5)
p2 = (5,9,8,6,2,4,1,3,7)

# Sanity check (lecture 3, slide 63)
p1 = (1,2,3,4,5,6,7,8,9)
p2 = (9,3,7,8,2,6,5,1,4)

# cycle_crossover(p1, p2)
child1, child2 = cycle_crossover(p1, p2)
print(f'{child1 = }')
print(f'{child2 = }')

child1 = array([1, 3, 7, 4, 2, 6, 5, 8, 9])
child2 = array([9, 2, 3, 8, 5, 6, 7, 1, 4])


#### Test crossovers

In [27]:
a = [2, 4, 7, 1, 3, 6, 8, 9, 5]
b = [5, 9, 8, 6, 2, 4, 1, 3, 7]
c, d = pmx(a, b)
e, f = order_crossover(a, b)
g, h = cycle_crossover(a, b)
print(f"Parents: {a} and {b}")
print(f"Children after PMX: {c} and {d}")
print(f"Children after Order Crossover: {e} and {f}")
print(f"Children after Cycle Crossover: {g} and {h}")

Parents: [2, 4, 7, 1, 3, 6, 8, 9, 5] and [5, 9, 8, 6, 2, 4, 1, 3, 7]
Children after PMX: [5 9 7 1 3 6 8 2 4] and [2 4 7 8 9 6 1 3 5]
Children after Order Crossover: [8 6 2 1 3 4 7 5 9] and [7 9 8 6 2 4 1 3 5]
Children after Cycle Crossover: [2 4 7 1 3 6 8 9 5] and [5 9 8 6 2 4 1 3 7]
